<h1>Instructions</h1>
<ol>
<li>Click the button "Not Trusted" on the menu bar of this notebook (at the top-right), and change the value to "Trusted". 
<li>Click Cell -> Run All. If you skip this step you might get the error "Cell magic `%%sql` not found."
<li>In the cell below each question, write a SQL query for the question.  
<li>After you enter a query, press Shift + Enter to run the cell. 
<li>After execution, the system will output the query result and say "CORRECT" if the query works for the sample dataset. Otherwise, it will say "INCORRECT" and also display the expected result for your comparison.
<li>Your submission will be tested over a dataset different from and larger than the sample. You will receive full credit for a question if the query returns correct results on the test dataset. 
</ol>

<h2>Notes</h2>
<ul>
<li>The DBMS running on the server is PostgreSQL 8.4.20. You may write any query to solve the questions as long as it conforms to its syntax. 
<li>You are encouraged to experiment with different queries by creating new code cells with the %%sql tag. However, when you submit your notebook, please make sure that there is exactly one cell with "%%sql n", for $1 \le n \le 12$, so that the autograder works properly. 
<li>Do not modify the line with %%sql; without it, the cell contents will be treated as Python code. 
</ul>

<h2>Example</h2>

As an example, you can copy and paste the following SQL code into the input box for one of the questions, and then press Shift + Enter. 

If the notebook is working properly, then you should see a table of all the bars in the database. If not, please contact the TA's or make a post on Piazza and we will do our best to resolve the issue. 

<h1>Database Schema</h1>

Drinkers(<u>name</u>, <u>addr</u>, hobby, frequent)

Bars(<u>name</u>, addr, owner)

Beers(<u>name</u>, brewer, alcohol)

Drinks(<u>drinkerName</u>, <u>drinkerAddr</u>, <u>beerName</u>, rating)

Sells(<u>barName</u>, <u>beerName</u>, price, discount)

Favorites(<u>drinkerName</u>, <u>drinkerAddr</u>, <u>barName</u>, beerName, season)

In [88]:
from IPython.core.magic import  (
    Magics, magics_class, cell_magic, line_magic
)
from IPython.display import clear_output, display, Markdown

import IPython.core.display as dis
import requests, json
import urllib.parse

server_url = "http://forward.cs.illinois.edu:8443"

def execQueryEval(query, query_id):
    cmd = server_url + "/test?query~"+urllib.parse.quote(query)+"&nb~"+str(query_id)
    cmd.encode("utf-8")
    r = requests.get(cmd)
    return r.text

def execQueryResult(query):
    cmd =  server_url + "/result?query~"+urllib.parse.quote(query)
    cmd.encode("utf-8")
    r = requests.get(cmd)
    return r.text

def displayAsMarkdown(text):
    display(Markdown(text))

@magics_class
class SQL(Magics):

    @cell_magic
    def sql(self, params, cell):
        clear_output()
        
        # remove trailing semicolon
        if cell.endswith(';'):
            cell = cell.rstrip(';')
            
        # get the question number
        n = -1
        if len(params.strip()) > 0:
            try:
                n = int(params.strip())
            except Exception:
                pass
        
        # evaluate the query
        if len(cell.strip()) > 1:
            print ("Query output: ")
            displayAsMarkdown(execQueryResult(cell))
            if n > -1:
                print ("Result: ")
                displayAsMarkdown(execQueryEval(cell, n))

## use ipython load_ext mechanism here if distributed
get_ipython().register_magics(SQL)

# hide this code cell
html = """
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input:eq(0)').hide();
    } else {
      $('div.input:eq(0)').show();
    }
    code_shown = !code_shown;
  }
  
  code_shown=true;
  code_toggle();
</script>
"""
dis.display_html(html, raw=True)

<h2>Question 1:</h2><br>
In California it is illegal to sell beverages with >60% alcohol content. Find the beers in the database that are illegal to sell in California. <br>

<b>Note</b> - alcohol concentration is represented as a floating point number between 0 and 1.

In [89]:
%%sql 1

/* write your query here */
SELECT Beers.name
FROM Beers
WHERE Beers.alcohol > 0.6

Query output: 


|name varchar|
|---|
|Armageddon|
|Snake Venom|
|Lucid Absinthe|
|Pure alcohol|
|Dangerous beer|


Result: 


CORRECT

Expected result: 

|name varchar|
|---|
|Armageddon|
|Snake Venom|
|Lucid Absinthe|
|Pure alcohol|
|Dangerous beer|


<h2>Question 2:</h2><br>
Find the name and location of bars that sell beers made by the 'Boston Beer Co.'.

In [90]:
%%sql 2

/* write your query here */
SELECT DISTINCT B.name, B.addr
FROM Bars as B,Sells as S, Beers as B1
WHERE B1.brewer = 'Boston Beer Co.'and S.beerName = B1.name and S.barName = B.name

Query output: 


|name varchar|addr varchar|
|---|---|
|Joe's Brewery|706 5th St|
|Green Bar 2|123 Green St|
|Boston Bar|100 Boston St|
|Legends|522 Green St|
|Brothers Bar & Grill|613 Green St|
|Radio Maria|119 Walnut St|
|The Red Lion|211 Green St|
|Cowboy Monkey|6 Taylor St|
|Jose's Brewery|706 6th St|
|Green Bar|123 Green St|
|Blind Pig Co.|120 Walnut St|
|Firehaus|708 6th St|
|Sipyard|204 Main St|
|Murphy's Pub|604 Green St|


Result: 


CORRECT

Expected result: 

|name varchar|addr varchar|
|---|---|
|Joe's Brewery|706 5th St|
|Green Bar 2|123 Green St|
|Boston Bar|100 Boston St|
|Legends|522 Green St|
|Brothers Bar & Grill|613 Green St|
|Radio Maria|119 Walnut St|
|The Red Lion|211 Green St|
|Cowboy Monkey|6 Taylor St|
|Jose's Brewery|706 6th St|
|Green Bar|123 Green St|
|Blind Pig Co.|120 Walnut St|
|Firehaus|708 6th St|
|Sipyard|204 Main St|
|Murphy's Pub|604 Green St|


<h2>Question 3:</h2><br>
Find all of the different beers that are sold in bars located on 'Green St'. <br>

<b>Note</b> - you can assume that address is always in the format '[number] [street]' <br>


In [91]:
%%sql 3

/* write your query here */
SELECT DISTINCT Sells.beerName
FROM   Bars, Sells 
WHERE Bars.addr like '%Green St'and Bars.name = Sells.barName

Query output: 


|beername varchar|
|---|
|Miller Lite|
|Sam Adams|
|Blue Moon|
|Boston flagship beer|
|Coors Light|
|Michelob|
|Heineken|
|Dangerous beer|
|Bud Light|
|Blue Ribbon|
|Lucid Absinthe|
|Budweiser|
|Guinness Draught|
|Armageddon|


Result: 


CORRECT

Expected result: 

|beername varchar|
|---|
|Miller Lite|
|Sam Adams|
|Blue Moon|
|Boston flagship beer|
|Coors Light|
|Michelob|
|Heineken|
|Dangerous beer|
|Bud Light|
|Blue Ribbon|
|Lucid Absinthe|
|Budweiser|
|Guinness Draught|
|Armageddon|


<h2>Question 4:</h2><br>
Find pairs of drinkers (name1, addr1, name2, addr2) who share a favorite drink and who frequent the same bar.<br>

<b>Note</b> - the context of each person’s favorite drink (barName and season) do not need to be the same.<br>

<b>Note</b> - it is expected that name1 || addr1 > name2 || addr2, so that the same information does not appear twice.

In [92]:
%%sql 4

/* write your query here */
SELECT DISTINCT d1.name,d1.addr, d2.name, d2.addr
FROM   drinkers as d1, drinkers as d2, Favorites as F1, Favorites as F2
WHERE d1.name||d1.addr > d2.name||d2.addr and d1.frequent = d2.frequent and d1.name = F1.drinkerName and d1.addr = F1.drinkerAddr and d2.addr = F2. drinkerAddr and d2.name = F2.drinkerName and F1.barName = F2.barName

Query output: 


|name varchar|addr varchar|name varchar|addr varchar|
|---|---|---|---|
|Nina|1400 Nectarine Ln|Ian|900 Ivy Ave|
|Ian|900 Ivy Ave|David|400 Durian Dr|
|Nina|1400 Nectarine Ln|David|400 Durian Dr|
|Frank|600 Fig St|Bob|200 Broccoli Blvd|
|Octavian|1500 Orange Dr|Mason|1300 Mulberry Ct|


Result: 


CORRECT

Expected result: 

|name varchar|addr varchar|name varchar|addr varchar|
|---|---|---|---|
|Nina|1400 Nectarine Ln|Ian|900 Ivy Ave|
|Nina|1400 Nectarine Ln|David|400 Durian Dr|
|Ian|900 Ivy Ave|David|400 Durian Dr|
|Frank|600 Fig St|Bob|200 Broccoli Blvd|
|Octavian|1500 Orange Dr|Mason|1300 Mulberry Ct|


<h2>Question 5:</h2><br>
Find the undiscounted price at which each bar sells the 'Sam Adams' beer

<b>Note</b> - the Sells.price attribute describes the price <b>after</b> all discounts are applied.<br>

<b>Note</b> - the discount attribute is a number between 0 and 1, such that discountedPrice = undiscountedPrice * discount

In [93]:
%%sql 5

/* write your query here */
SELECT barName, price / discount
FROM Sells
WHERE Sells.beerName = 'Sam Adams'

Query output: 


|barname varchar|?column? numeric|
|---|---|
|Blind Pig Co.|7.5000000000000000|
|Brothers Bar & Grill|7.5000000000000000|
|Firehaus|7.5000000000000000|
|Joe's Brewery|10.0000000000000000|
|Legends|13.0000000000000000|
|Murphy's Pub|7.5000000000000000|
|Radio Maria|7.0000000000000000|
|The Red Lion|7.5000000000000000|
|Sipyard|8.5000000000000000|
|Cowboy Monkey|5.0000000000000000|
|Boston Bar|10.0000000000000000|
|Green Bar|25.0000000000000000|
|Green Bar 2|15.0000000000000000|
|Jose's Brewery|10.0000000000000000|


Result: 


CORRECT

Expected result: 

|barname varchar|undiscountedprice numeric|
|---|---|
|Blind Pig Co.|7.5000000000000000|
|Brothers Bar & Grill|7.5000000000000000|
|Firehaus|7.5000000000000000|
|Joe's Brewery|10.0000000000000000|
|Legends|13.0000000000000000|
|Murphy's Pub|7.5000000000000000|
|Radio Maria|7.0000000000000000|
|The Red Lion|7.5000000000000000|
|Sipyard|8.5000000000000000|
|Cowboy Monkey|5.0000000000000000|
|Boston Bar|10.0000000000000000|
|Green Bar|25.0000000000000000|
|Green Bar 2|15.0000000000000000|
|Jose's Brewery|10.0000000000000000|


<h2>Question 6:</h2><br>
Find the bars that sell a unique beer (a beer that no other bars have)

In [94]:
%%sql 6

/* write your query here */
SELECT Sells.barName
FROM Sells
WHERE Sells.beerName not in 
(SELECT DISTINCT s1.beerName
FROM Sells as s1, Sells as s2
WHERE s1.beerName = s2.beerName and s1.barName <> s2.barName)

Query output: 


|barname varchar|
|---|
|Brothers Bar & Grill|
|Cowboy Monkey|
|Firehaus|
|Sipyard|
|Sipyard|
|Boston Bar|
|Murphy's Pub|


Result: 


CORRECT

Expected result: 

|barname varchar|
|---|
|Boston Bar|
|Murphy's Pub|
|Cowboy Monkey|
|Sipyard|
|Brothers Bar & Grill|
|Firehaus|


<h2>Question 7:</h2><br>
Find the 5 most popular beers (according to the total number of people who drink the beer across all bars) that are sold at 'Murphy''s Pub'.

In [145]:
%%sql 7

/* write your query here */
SELECT *
FROM (
SELECT a1.new_b_n
FROM (SELECT Drinks.beerName as new_b_n, COUNT(drinkerName) as count_num
FROM Drinks
WHERE Drinks.beerName in (
SELECT Sells.beerName
FROM Sells
WHERE Sells.barName = 'Murphy''s Pub')
GROUP BY Drinks.beerName) as a1
ORDER BY a1.count_num DESC) as c1
LIMIT 5


Query output: 


|new_b_n varchar|
|---|
|Guinness Draught|
|Sam Adams|
|Blue Moon|
|Boston flagship beer|
|Heineken|


Result: 


CORRECT

Expected result: 

|beername varchar|
|---|
|Guinness Draught|
|Boston flagship beer|
|Sam Adams|
|Heineken|
|Blue Moon|


<h2>Question 8:</h2><br>
Find the bars that serve the Sam Adams beer cheaper than the average market price.

<b>Note</b> - the average market price is defined as the average price of the beer across all bars that sell the beer. 

In [96]:
%%sql 8

/* write your query here */
SELECT list2.barName
FROM (SELECT AVG(Sells.price) as avg_all
    FROM Sells
    WHERE Sells.beerName = 'Sam Adams') as list1 , ( SELECT Sells.barName,AVG(Sells.price) as ind_avg
    FROM Sells
    WHERE Sells.beerName = 'Sam Adams'
    GROUP BY Sells.barName) as list2
WHERE list2.ind_avg < list1.avg_all

Query output: 


|barname varchar|
|---|
|Joe's Brewery|
|Blind Pig Co.|
|Boston Bar|
|Legends|
|Murphy's Pub|
|Firehaus|
|Radio Maria|
|Cowboy Monkey|
|Brothers Bar & Grill|
|Jose's Brewery|
|The Red Lion|


Result: 


CORRECT

Expected result: 

|barname varchar|
|---|
|Blind Pig Co.|
|Brothers Bar & Grill|
|Firehaus|
|Joe's Brewery|
|Legends|
|Murphy's Pub|
|Radio Maria|
|The Red Lion|
|Cowboy Monkey|
|Boston Bar|
|Jose's Brewery|


<h2>Question 9:</h2><br>
Find pairs of bars (Name1, Name2) that sell the same set of drinks. <br>

<b>Note</b> - for each pair, we would like for Name1 > Name2, so that the same information does not appear twice.

In [97]:
%%sql 9

/* write your query here */
SELECT DISTINCT S3.barName, S4.barName
FROM Sells as S3, Sells as S4
WHERE (S3.barName > S4.barName) and ((SELECT COUNT(beerName)
                                     FROM Sells
                                     WHERE Sells.barName = S3.barName) = (SELECT COUNT(beerName)
                                     FROM Sells
                                     WHERE Sells.barName = S4.barName)) and (S3.barName,S4.barName) NOT IN (SELECT DISTINCT S1.barName, S2.barName
                    FROM Sells as S1, Sells as S2
                    WHERE S1.barName > S2.barName and S1.beerName NOT IN (SELECT Sells.beerName
                                                                          FROM Sells
                                                                          WHERE Sells.barName = S2.barName))                           

Query output: 


|barname varchar|barname varchar|
|---|---|
|Joe's Brewery|Blind Pig Co.|
|Jose's Brewery|Blind Pig Co.|
|Jose's Brewery|Joe's Brewery|
|Green Bar 2|Green Bar|


Result: 


CORRECT

Expected result: 

|name varchar|name varchar|
|---|---|
|Joe's Brewery|Blind Pig Co.|
|Jose's Brewery|Blind Pig Co.|
|Jose's Brewery|Joe's Brewery|
|Green Bar 2|Green Bar|


<h2>Question 10:</h2><br>
Suppose we are interested in computing statistics about the ratings of people’s favorite beers. Obtain a list of all ratings for all beers that have been favorited by >= 3 different people. 

In [98]:
%%sql 10

/* write your query here */
SELECT Drinks.rating
FROM Drinks
WHERE Drinks.beerName IN (SELECT beerName
FROM Favorites 
GROUP BY beerName
HAVING COUNT(drinkerName) >= 3)




Query output: 


|rating int4|
|---|
|8|
|7|
|8|
|8|
|7|
|8|
|8|
|3|
|8|
|8|
|8|
|9|
|7|
|6|
|7|
|10|
|5|
|6|
|5|
|7|
|10|
|4|
|3|


Result: 


CORRECT

Expected result: 

|rating int4|
|---|
|8|
|7|
|8|
|8|
|7|
|8|
|3|
|8|
|8|
|8|
|9|
|6|
|7|
|10|
|5|
|6|
|5|
|7|
|10|
|4|
|3|


<h2>Question 11:</h2><br>
Find the most expensive beer, along with its price, that is sold at each bar. <br>

<b>Note</b> - the expected output format is (bar name, name of most expensive beer at that bar, price of most expensive beer) <br>

<b>Note</b> - you can assume that there will be at most one row for each bar.

In [99]:
%%sql 11

/* write your query here */
SELECT Sells.barName, Sells.beerName, Sells.price
FROM Sells, (SELECT barName, MAX(price) as max1
FROM Sells
GROUP BY barName) as S1
WHERE Sells.barName = S1.barName and Sells.price = S1.max1 

Query output: 


|barname varchar|beername varchar|price numeric|
|---|---|---|
|Blind Pig Co.|Corona|8.00|
|Brothers Bar & Grill|Blue Ribbon|9.50|
|The Clybourne|Heineken|10.00|
|Cowboy Monkey|Lucid Absinthe|95.00|
|Firehaus|Snake Venom|19.50|
|Joe's Brewery|Budweiser|7.50|
|Kam's|Corona|8.00|
|Legends|Armageddon|49.50|
|Radio Maria|Sam Adams|7.00|
|The Red Lion|Guinness Draught|9.50|
|Sipyard|Stella Artois|39.50|
|Boston Bar|Boston flagship beer|6.00|
|Green Bar|Armageddon|101.00|
|Green Bar 2|Lucid Absinthe|100.01|
|Murphy's Pub|Dangerous beer|999.00|
|Jose's Brewery|Budweiser|7.50|


Result: 


CORRECT

Expected result: 

|barname varchar|beername varchar|price numeric|
|---|---|---|
|Blind Pig Co.|Corona|8.00|
|Brothers Bar & Grill|Blue Ribbon|9.50|
|The Clybourne|Heineken|10.00|
|Cowboy Monkey|Lucid Absinthe|95.00|
|Firehaus|Snake Venom|19.50|
|Joe's Brewery|Budweiser|7.50|
|Kam's|Corona|8.00|
|Legends|Armageddon|49.50|
|Radio Maria|Sam Adams|7.00|
|The Red Lion|Guinness Draught|9.50|
|Sipyard|Stella Artois|39.50|
|Boston Bar|Boston flagship beer|6.00|
|Green Bar|Armageddon|101.00|
|Green Bar 2|Lucid Absinthe|100.01|
|Murphy's Pub|Dangerous beer|999.00|
|Jose's Brewery|Budweiser|7.50|


<h2>Question 12:</h2><br>
Find the beer with the highest average rating among the beers with the lowest alcohol content. <br>

<b>Note</b> - you may assume that the output will only contain one row.

In [100]:
%%sql 12

/* write your query here */

SELECT b1.beerName
FROM (
SELECT beerName, AVG(rating) as avg_r
FROM Drinks
WHERE Drinks.beerName IN(
SELECT Beers.name 
FROM Beers
WHERE Beers.alcohol = (SELECT MIN(Beers.alcohol)
FROM Beers))
GROUP BY Drinks.beerName
) as b1
WHERE b1.avg_r = (
SELECT MAX(a1.avg_r)
FROM (SELECT beerName, AVG(rating) as avg_r
FROM Drinks
WHERE Drinks.beerName IN(
SELECT Beers.name 
FROM Beers
WHERE Beers.alcohol = (SELECT MIN(Beers.alcohol)
FROM Beers))
GROUP BY Drinks.beerName) as a1
)

Query output: 


|beername varchar|
|---|
|Paul Revere ale|


Result: 


CORRECT

Expected result: 

|beername varchar|
|---|
|Paul Revere ale|
